In [ ]:
# Comparación: Chat Completions vs Responses API
# Ejemplo práctico de migración para mantener historial de conversación

import os
from dotenv import load_dotenv
from openai import OpenAI

# Cargar variables de entorno
load_dotenv()

# Configurar cliente con token desde .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# =============================================================================
# MÉTODO TRADICIONAL (Chat Completions) - Gestión manual del historial
# =============================================================================

print("=== MÉTODO TRADICIONAL (Chat Completions) ===")
messages = [{"role": "system", "content": "You are a helpful math tutor that speaks concisely."}]
user_msgs = ["Explain what pi is.", "Summarize this in two bullet points."]

# Loop over the user questions
for q in user_msgs:
    print("User: ", q)
    
    # Create a dictionary for the user message from q and append to messages
    user_dict = {"role": 'user', "content": q}
    messages.append(user_dict)
    
    # Create the API request
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_completion_tokens=100
    )
    
    # Append the assistant's message to messages
    assistant_dict = {"role": "assistant", "content": response.choices[0].message.content}
    messages.append(assistant_dict)
    print("Assistant: ", response.choices[0].message.content, "\n")

print("="*70)

In [ ]:
# =============================================================================
# NUEVO MÉTODO (Responses API) - Gestión automática del historial
# =============================================================================

print("\n=== NUEVO MÉTODO (Responses API) ===")

# Configurar prompt inicial del sistema
system_prompt = "You are a helpful math tutor that speaks concisely."
user_msgs = ["Explain what pi is.", "Summarize this in two bullet points."]

# Variable para almacenar el ID de la conversación
conversation_id = None

# Loop over the user questions
for i, q in enumerate(user_msgs):
    print("User: ", q)
    
    if i == 0:
        # Primera consulta: incluir sistema + usuario
        input_text = f"System: {system_prompt}\n\nUser: {q}"
        
        # Crear primera respuesta con store=True para mantener estado
        response = client.responses.create(
            model="gpt-4o-mini",
            input=input_text,
            store=True  # Habilita gestión automática de estado
        )
        
        # Guardar ID para continuar conversación
        conversation_id = response.id
        
    else:
        # Consultas siguientes: usar previous_response_id para mantener historial
        response = client.responses.create(
            model="gpt-4o-mini",
            input=q,
            previous_response_id=conversation_id,
            store=True
        )
        
        # Actualizar conversation_id para próxima interacción
        conversation_id = response.id
    
    print("Assistant: ", response.content, "\n")

print("="*70)

In [ ]:
# =============================================================================
# EJEMPLO PRÁCTICO: Aplicando Responses API al Chatbot AB Testing
# =============================================================================

print("\n=== CHATBOT AB TESTING CON RESPONSES API ===")

class ABTestingChatbotResponses:
    """Chatbot AB Testing usando la nueva API Responses"""
    
    def __init__(self):
        # Usar el mismo cliente ya configurado con .env
        self.client = client
        self.conversations = {}  # Almacenar IDs por sesión
        
    def chat(self, query: str, session_id: str = "default"):
        """Envía una consulta y mantiene historial automáticamente"""
        
        # Prompt del sistema para AB Testing
        system_prompt = """Eres un analista experto en AB Testing. 
        Responde preguntas sobre datos de experimentos, métricas de conversión, 
        y análisis estadístico de manera concisa y precisa."""
        
        # Verificar si existe conversación previa
        previous_id = self.conversations.get(session_id)
        
        if previous_id:
            # Continuar conversación existente
            response = self.client.responses.create(
                model="gpt-4o-mini",
                input=query,
                previous_response_id=previous_id,
                store=True
            )
        else:
            # Iniciar nueva conversación
            input_text = f"System: {system_prompt}\n\nUser: {query}"
            response = self.client.responses.create(
                model="gpt-4o-mini", 
                input=input_text,
                store=True
            )
        
        # Actualizar ID de conversación
        self.conversations[session_id] = response.id
        
        return response.content
    
    def reset_session(self, session_id: str = "default"):
        """Reinicia una sesión de conversación"""
        if session_id in self.conversations:
            del self.conversations[session_id]
            print(f"Sesión '{session_id}' reiniciada.")
    
    def get_conversation_history(self, session_id: str = "default"):
        """Obtiene el historial completo (funcionalidad de Responses API)"""
        conversation_id = self.conversations.get(session_id)
        if conversation_id:
            try:
                # Obtener respuesta completa con historial
                response = self.client.responses.retrieve(conversation_id)
                return response.content
            except Exception as e:
                return f"Error obteniendo historial: {e}"
        return "No hay conversación activa."

# Ejemplo de uso
chatbot = ABTestingChatbotResponses()

# Simulación de conversación con historial automático
queries = [
    "¿Qué es un AB Test?",
    "¿Cómo se calcula el lift?", 
    "Dame un ejemplo con números específicos"
]

print("Conversación con historial automático:")
for i, query in enumerate(queries, 1):
    print(f"\n[{i}] Usuario: {query}")
    response = chatbot.chat(query, session_id="demo")
    print(f"[{i}] Bot: {response}")

print("\n" + "="*70)

## Ventajas de la Migración a Responses API

### ✅ **Antes (Chat Completions)**
- ❌ Gestión manual del array `messages`
- ❌ Código más complejo para mantener historial
- ❌ Riesgo de perder contexto por errores de código
- ❌ Manejo manual de límites de tokens

### ✅ **Después (Responses API)**
- ✅ Gestión automática del historial con `previous_response_id`
- ✅ Código más limpio y mantenible
- ✅ Estado persistente manejado por OpenAI
- ✅ Optimización automática de tokens
- ✅ Soporte nativo para herramientas avanzadas

### 🔑 **Puntos Clave para la Migración**

1. **`store=True`**: Habilita la gestión automática de estado
2. **`previous_response_id`**: Continúa conversaciones existentes
3. **Menos código**: No necesitas gestionar el array de mensajes
4. **Mejor rendimiento**: OpenAI optimiza automáticamente el contexto